In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
%cd /content/drive/My\ Drive/nlp_data

/content/drive/.shortcut-targets-by-id/1TX7I-_KwAgP0OHczEL8zSwOuxJHxViff/nlp_data


In [3]:
!pip3 install pickle5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import pickle
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor
from torch.autograd import Variable
import torch.optim as optim
import warnings
import numpy as np
import torch.nn.functional as F
import pandas as pd
from sklearn.metrics import classification_report
from torch.utils.data.sampler import SubsetRandomSampler
import time
import pickle5 as pickle
from sklearn.utils.class_weight import compute_class_weight

warnings.filterwarnings("ignore")

In [5]:
if torch.cuda.is_available():
  device = torch.device("cuda")
  print("Using CUDA - GPU")
else:
  device = torch.device("cpu")

Using CUDA - GPU


In [6]:
# Read train BERT embeddings
with open("./embeddings/train_bert_embeddings.pkl", "rb") as f:
    training_data = pickle.load(f)

In [7]:
len(training_data['embeddings']), training_data['embeddings'][0].shape

(7531, torch.Size([1, 512, 768]))

In [8]:
# Read test BERT embeddings
with open("./embeddings/test_bert_embeddings.pkl", "rb") as f:
    testing_data = pickle.load(f)

In [9]:
train_dataset_path = "tos_clauses_train.csv"
test_dataset_path = "tos_clauses_dev.csv"

In [10]:
train_df = pd.read_csv(train_dataset_path, header=0)
test_df = pd.read_csv(test_dataset_path, header=0)


In [11]:
train_targets = train_df.label.values
test_targets = test_df.label.values


In [12]:
len(train_targets)

7531

In [13]:
class Dataset(object):
    """An abstract class representing a Dataset.
    All other datasets should subclass it. All subclasses should
    override ``__len__``, that provides the size of the dataset,
    and ``__getitem__``, supporting integer indexing in range
    from 0 to len(self) exclusive.
    """

    def __getitem__(self, index):
        raise NotImplementedError

    def __len__(self):
        raise NotImplementedError

    def __add__(self, other):
        return ConcatDataset([self, other])


In [14]:
class TOSDataset(Dataset):
    def __init__(self, X, Y, transform=None):
        self.data1 = X
        self.data2 = Y
        self.transform = transform

    def __len__(self):
        return len(self.data1)

    def __getitem__(self, index):
        x = self.data1[index]
        y = self.data2[index]

        if self.transform is not None:
            x = torch.tensor(x)

        return torch.squeeze(x, dim=1), torch.tensor(y)


In [15]:
test_len = len(test_df)
train_len = len(train_df)
X_train_tensor = TOSDataset(train_df["sentences"], train_df["label"])

num_train = len(X_train_tensor)
indices = list(range(num_train))
np.random.shuffle(indices)
train_sampler = SubsetRandomSampler(indices)
print(train_sampler)
train_df_by_index = train_df.loc[indices]
train_fair = sum(train_df_by_index["label"] == 0)
train_unfair = sum(train_df_by_index["label"] == 1)
print("train_fair:" + str(train_fair))
print("train_unfair:" + str(train_unfair))


train_fair:6705
train_unfair:826


In [16]:
train_data = TOSDataset(training_data["embeddings"], train_targets, transform=transforms.ToTensor())
test_data = TOSDataset(testing_data["embeddings"], test_targets, transform=transforms.ToTensor())


In [17]:
# how many samples per batch to load
BATCH_SIZE = 20

# number of subprocesses to use for data loading
NUM_WORKERS = 0


In [18]:
# prepare data loaders
train_loader = DataLoader(
    train_data, batch_size=BATCH_SIZE, sampler=train_sampler, num_workers=NUM_WORKERS
)
test_loader = DataLoader(
    test_data, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS
)


In [19]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [20]:
EMBEDDING_DIM = 768
HIDDEN_DIM = 512
OUTPUT_DIM = 2
N_EPOCHS = 50

In [21]:
class_weight = compute_class_weight(
    "balanced", classes=np.unique(train_df_by_index["label"]), y=train_df_by_index["label"]
)
class_weight

array([0.56159582, 4.55871671])

#**Simple RNN**

In [36]:
class RNNet(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):

        super(RNNet, self).__init__()

        # Number of hidden dimensions
        self.hidden_dim = hidden_dim

        # RNN
        self.rnn = nn.RNN(input_dim, hidden_dim, num_layers=3, batch_first=True, nonlinearity="relu")

        # Readout layer
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):

        # Initialize hidden state with zeros
        h0 = Variable(torch.zeros(3, x.size(0), self.hidden_dim)).to(device) # number of layers

        # One time step
        out, hn = self.rnn(x, h0)
        out = self.fc(out[:, -1, :])
        return out


In [39]:
model = RNNet(EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM).to(device)

loss_fn = nn.CrossEntropyLoss(weight=torch.FloatTensor(class_weight)).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-4)
test_min_loss = np.inf

for epoch in range(N_EPOCHS):

    start_time = time.time()
    model.train()
    train_loss = 0.0
    test_loss = 0.0
    for inputs, target in train_loader:
        inputs, target = inputs.to(device), target.to(device)
        model.zero_grad()
        inputs = torch.squeeze(inputs, dim=1)
        output = model(inputs)
        loss = loss_fn(output, target)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()

    y_pred_list = []
    y_targ_list = []
    model.eval()
    for inputs, target in test_loader:
        inputs, target = inputs.to(device), target.to(device)
        inputs = torch.squeeze(inputs, dim=1)
        output = model(inputs)
        loss = loss_fn(output, target)
        test_loss += loss.item()
        _, y_test_pred = torch.max(output, 1)
        y_pred_tag = y_test_pred
        y_pred_list.append(y_pred_tag.cpu().numpy())
        y_targ_list.append(target.cpu().numpy())
    

    if(epoch%1 == 0):
        y_pred_list = [x.squeeze().tolist() for x in y_pred_list]
        y_targ_list = [x.squeeze().tolist() for x in y_targ_list]
        y_pred_list = [x for sublist in y_pred_list for x in sublist]
        y_targ_list = [x for sublist in y_targ_list for x in sublist]

        print(classification_report(y_targ_list, y_pred_list))

    train_loss = train_loss / len(train_loader.dataset)
    test_loss = test_loss / len(test_loader.dataset)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    print(f"Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s")
    print("\tTraining Loss: {:.6f} \Test Loss: {:.6f}".format(train_loss, test_loss))
    if test_loss <= test_min_loss:
        print("Test loss decreased ({:.6f} --> {:.6f}). Saving model...".format(test_min_loss, test_loss))
        torch.save(model.state_dict(), "models/rnn_bert_model.pt")
        test_min_loss = test_loss
    print("="*70)


              precision    recall  f1-score   support

           0       0.93      0.60      0.73      1677
           1       0.17      0.66      0.27       206

    accuracy                           0.60      1883
   macro avg       0.55      0.63      0.50      1883
weighted avg       0.85      0.60      0.68      1883

Epoch: 01 | Epoch Time: 0m 24s
	Training Loss: 0.033579 \Test Loss: 0.033231
Test loss decreased (inf --> 0.033231). Saving model...
              precision    recall  f1-score   support

           0       0.96      0.34      0.51      1677
           1       0.14      0.88      0.24       206

    accuracy                           0.40      1883
   macro avg       0.55      0.61      0.38      1883
weighted avg       0.87      0.40      0.48      1883

Epoch: 02 | Epoch Time: 0m 24s
	Training Loss: 0.032284 \Test Loss: 0.032683
Test loss decreased (0.033231 --> 0.032683). Saving model...
              precision    recall  f1-score   support

           0       0

In [40]:
y_pred_list = []
y_targ_list = []
model = RNNet(EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM).to(device)
model.load_state_dict(torch.load("models/rnn_bert_model.pt"))
model.eval()

with torch.no_grad():
    for inputs, target in test_loader:
        inputs, target = inputs.to(device), target.to(device)
        inputs = torch.squeeze(inputs, dim=1)
        y_test_pred = model(inputs)
        _, y_test_pred = torch.max(y_test_pred, 1)
        y_pred_tag = y_test_pred
        y_pred_list.append(y_pred_tag.cpu().numpy())
        y_targ_list.append(target.cpu().numpy())

y_pred_list = [x.squeeze().tolist() for x in y_pred_list]
y_targ_list = [x.squeeze().tolist() for x in y_targ_list]
y_pred_list = [x for sublist in y_pred_list for x in sublist]
y_targ_list = [x for sublist in y_targ_list for x in sublist]

print(classification_report(y_targ_list, y_pred_list))

              precision    recall  f1-score   support

           0       0.95      0.74      0.83      1677
           1       0.24      0.67      0.35       206

    accuracy                           0.73      1883
   macro avg       0.59      0.70      0.59      1883
weighted avg       0.87      0.73      0.78      1883



#**Gated RNN**

In [27]:
class GRU_Network(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):

        super(GRU_Network, self).__init__()

        # Number of hidden dimensions
        self.hidden_dim = hidden_dim

        # RNN
        self.rnn = nn.GRU(input_dim, hidden_dim, num_layers=1, batch_first=True)

        # Readout layer
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):

        # Initialize hidden state with zeros
        h0 = Variable(torch.zeros(1, x.size(0), self.hidden_dim)).to(device)

        # One time step
        out, hn = self.rnn(x, h0)
        out = self.fc(out[:, -1, :])
        return out


In [28]:
model_gru = GRU_Network(EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM).to(device)
test_min_loss = np.inf
criterion = nn.CrossEntropyLoss(weight=torch.FloatTensor(class_weight)).to(device)
optimizer_gru = optim.Adam(model_gru.parameters(), lr=1e-3)
    
for epoch in range(N_EPOCHS):
    start_time = time.time()
    model_gru.train()
    train_loss = 0.0
    test_loss = 0.0
    for inputs, target in train_loader:
        inputs, target = inputs.to(device), target.to(device)
        model_gru.zero_grad()
        inputs = torch.squeeze(inputs, dim=1)
        output = model_gru(inputs)
        loss = criterion(output, target)
        loss.backward()
        train_loss += loss.item()
        optimizer_gru.step()

    y_pred_list = []
    y_targ_list = []
    model_gru.eval()
    for inputs, target in test_loader:
        inputs, target = inputs.to(device), target.to(device)
        inputs = torch.squeeze(inputs, dim=1)
        output = model_gru(inputs)
        loss = criterion(output, target)
        test_loss += loss.item()
        _, y_test_pred = torch.max(output, 1)
        y_pred_tag = y_test_pred
        y_pred_list.append(y_pred_tag.cpu().numpy())
        y_targ_list.append(target.cpu().numpy())
    

    if(epoch%1 == 0):
        y_pred_list = [x.squeeze().tolist() for x in y_pred_list]
        y_targ_list = [x.squeeze().tolist() for x in y_targ_list]
        y_pred_list = [x for sublist in y_pred_list for x in sublist]
        y_targ_list = [x for sublist in y_targ_list for x in sublist]

        print(classification_report(y_targ_list, y_pred_list))

    train_loss = train_loss / len(train_loader.dataset)
    test_loss = test_loss / len(test_loader.dataset)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    print(f"Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s")
    print("\tTraining Loss: {:.6f} \Test Loss: {:.6f}".format(train_loss, test_loss))
    if test_loss <= test_min_loss:
        print("Test loss decreased ({:.6f} --> {:.6f}). Saving model...\n".format(test_min_loss, test_loss))
        torch.save(model_gru.state_dict(), "./models/gru_bert_model.pt")
        test_min_loss = test_loss
    print("="*70)


              precision    recall  f1-score   support

           0       0.94      0.85      0.89      1677
           1       0.32      0.58      0.41       206

    accuracy                           0.82      1883
   macro avg       0.63      0.71      0.65      1883
weighted avg       0.87      0.82      0.84      1883

Epoch: 01 | Epoch Time: 0m 15s
	Training Loss: 0.034934 \Test Loss: 0.029151
Test loss decreased (inf --> 0.029151). Saving model...

              precision    recall  f1-score   support

           0       0.92      0.97      0.94      1677
           1       0.57      0.31      0.40       206

    accuracy                           0.90      1883
   macro avg       0.74      0.64      0.67      1883
weighted avg       0.88      0.90      0.88      1883

Epoch: 02 | Epoch Time: 0m 15s
	Training Loss: 0.028747 \Test Loss: 0.031566
              precision    recall  f1-score   support

           0       0.95      0.91      0.93      1677
           1       0.44   

In [29]:
y_pred_list = []
y_targ_list = []
model = GRU_Network(EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM).to(device)
model.load_state_dict(torch.load("models/gru_bert_model.pt"))
model.eval()

with torch.no_grad():
    for inputs, target in test_loader:
        inputs, target = inputs.to(device), target.to(device)
        inputs = torch.squeeze(inputs, dim=1)
        y_test_pred = model(inputs)
        _, y_test_pred = torch.max(y_test_pred, 1)
        y_pred_tag = y_test_pred
        y_pred_list.append(y_pred_tag.cpu().numpy())
        y_targ_list.append(target.cpu().numpy())

y_pred_list = [x.squeeze().tolist() for x in y_pred_list]
y_targ_list = [x.squeeze().tolist() for x in y_targ_list]
y_pred_list = [x for sublist in y_pred_list for x in sublist]
y_targ_list = [x for sublist in y_targ_list for x in sublist]

print(classification_report(y_targ_list, y_pred_list))

              precision    recall  f1-score   support

           0       0.97      0.84      0.90      1677
           1       0.37      0.76      0.50       206

    accuracy                           0.83      1883
   macro avg       0.67      0.80      0.70      1883
weighted avg       0.90      0.83      0.86      1883



#**LSTM**

In [30]:
# LSTM

class LSTM_Network(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        
        super(LSTM_Network, self).__init__()

         # Number of hidden dimensions
        self.hidden_dim = hidden_dim
        
        # RNN
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers=1, batch_first=True)
        
        # Readout layer
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        
        # Initialize hidden state with zeros
        h0 = Variable(torch.zeros(1, x.size(0), self.hidden_dim)).to(device)
        c0 = Variable(torch.zeros(1, x.size(0), self.hidden_dim)).to(device)
        
        # One time step
        out, (hn, cn) = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        
        return out

In [31]:
model_lstm = LSTM_Network(EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM).to(device)
test_min_loss = np.inf
criterion = nn.CrossEntropyLoss(weight=torch.FloatTensor(class_weight)).to(device)
optimizer_lstm = optim.Adam(model_lstm.parameters(), lr=1e-3)
    
for epoch in range(N_EPOCHS):
    start_time = time.time()
    model_lstm.train()
    train_loss = 0.0
    test_loss = 0.0
    for inputs, target in train_loader:
        inputs, target = inputs.to(device), target.to(device)
        model_lstm.zero_grad()
        inputs = torch.squeeze(inputs, dim=1)
        output = model_lstm(inputs)
        loss = criterion(output, target)
        loss.backward()
        train_loss += loss.item()
        optimizer_lstm.step()

    y_pred_list = []
    y_targ_list = []
    model_lstm.eval()
    for inputs, target in test_loader:
        inputs, target = inputs.to(device), target.to(device)
        inputs = torch.squeeze(inputs, dim=1)
        output = model_lstm(inputs)
        loss = criterion(output, target)
        test_loss += loss.item()
        _, y_test_pred = torch.max(output, 1)
        y_pred_tag = y_test_pred
        y_pred_list.append(y_pred_tag.cpu().numpy())
        y_targ_list.append(target.cpu().numpy())
    

    if(epoch%1 == 0):
        y_pred_list = [x.squeeze().tolist() for x in y_pred_list]
        y_targ_list = [x.squeeze().tolist() for x in y_targ_list]
        y_pred_list = [x for sublist in y_pred_list for x in sublist]
        y_targ_list = [x for sublist in y_targ_list for x in sublist]

        print(classification_report(y_targ_list, y_pred_list))

    train_loss = train_loss / len(train_loader.dataset)
    test_loss = test_loss / len(test_loader.dataset)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    print(f"Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s")
    print("\tTraining Loss: {:.6f} | Test Loss: {:.6f}".format(train_loss, test_loss))
    if test_loss <= test_min_loss:
        print("Test loss decreased ({:.6f} --> {:.6f}). Saving model...\n".format(test_min_loss, test_loss))
        torch.save(model_lstm.state_dict(), "models/lstm_bert_model.pt")
        test_min_loss = test_loss
    print("="*70)


              precision    recall  f1-score   support

           0       0.92      0.86      0.89      1677
           1       0.24      0.35      0.28       206

    accuracy                           0.80      1883
   macro avg       0.58      0.61      0.58      1883
weighted avg       0.84      0.80      0.82      1883

Epoch: 01 | Epoch Time: 0m 15s
	Training Loss: 0.033822 | Test Loss: 0.032260
Test loss decreased (inf --> 0.032260). Saving model...

              precision    recall  f1-score   support

           0       0.91      0.94      0.93      1677
           1       0.34      0.24      0.28       206

    accuracy                           0.87      1883
   macro avg       0.62      0.59      0.60      1883
weighted avg       0.85      0.87      0.86      1883

Epoch: 02 | Epoch Time: 0m 15s
	Training Loss: 0.032458 | Test Loss: 0.032388
              precision    recall  f1-score   support

           0       0.93      0.77      0.84      1677
           1       0.21 

In [32]:
y_pred_list = []
y_targ_list = []
model = LSTM_Network(EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM).to(device)
model.load_state_dict(torch.load("models/lstm_bert_model.pt"))
model.eval()

with torch.no_grad():
    for inputs, target in test_loader:
        inputs, target = inputs.to(device), target.to(device)
        inputs = torch.squeeze(inputs, dim=1)
        y_test_pred = model(inputs)
        _, y_test_pred = torch.max(y_test_pred, 1)
        y_pred_tag = y_test_pred
        y_pred_list.append(y_pred_tag.cpu().numpy())
        y_targ_list.append(target.cpu().numpy())

y_pred_list = [x.squeeze().tolist() for x in y_pred_list]
y_targ_list = [x.squeeze().tolist() for x in y_targ_list]
y_pred_list = [x for sublist in y_pred_list for x in sublist]
y_targ_list = [x for sublist in y_targ_list for x in sublist]

print(classification_report(y_targ_list, y_pred_list))

              precision    recall  f1-score   support

           0       0.95      0.68      0.79      1677
           1       0.22      0.74      0.34       206

    accuracy                           0.69      1883
   macro avg       0.59      0.71      0.57      1883
weighted avg       0.87      0.69      0.74      1883



#**Bi-LSTM**

In [33]:
class Bi_LSTM_Network(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):

        super(Bi_LSTM_Network, self).__init__()

        # Number of hidden dimensions
        self.hidden_dim = hidden_dim

        # RNN
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers=1, batch_first=True, bidirectional=True)

        # Readout layer
        self.fc = nn.Linear(hidden_dim * 2, output_dim)

    def forward(self, x):

        # Initialize hidden state with zeros
        h0 = Variable(torch.zeros(1 * 2, x.size(0), self.hidden_dim)).to(device)
        c0 = Variable(torch.zeros(1 * 2, x.size(0), self.hidden_dim)).to(device)

        # One time step
        out, (hn, cn) = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])

        return out


In [34]:
model_bilstm = Bi_LSTM_Network(EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM).to(device)
test_min_loss = np.inf
criterion = nn.CrossEntropyLoss(weight=torch.FloatTensor(class_weight)).to(device)
optimizer_bilstm = optim.Adam(model_bilstm.parameters(), lr=1e-3) 
    
for epoch in range(N_EPOCHS):
    start_time = time.time()
    model_bilstm.train()
    train_loss = 0.0
    test_loss = 0.0
    for inputs, target in train_loader:
        inputs, target = inputs.to(device), target.to(device)
        optimizer_bilstm.zero_grad()
        inputs = torch.squeeze(inputs, dim=1)
        output = model_bilstm(inputs)
        loss = criterion(output, target)
        loss.backward()
        train_loss += loss.item()
        # torch.nn.utils.clip_grad_norm_(model_bilstm.parameters(), max_norm=4.0, norm_type=2) # try
        optimizer_bilstm.step()

    y_pred_list = []
    y_targ_list = []
    model_bilstm.eval()
    for inputs, target in test_loader:
        inputs, target = inputs.to(device), target.to(device)
        inputs = torch.squeeze(inputs, dim=1)
        output = model_bilstm(inputs)
        loss = criterion(output, target)
        test_loss += loss.item()
        _, y_test_pred = torch.max(output, 1)
        y_pred_tag = y_test_pred
        y_pred_list.append(y_pred_tag.cpu().numpy())
        y_targ_list.append(target.cpu().numpy())
    

    if(epoch%1 == 0):
        y_pred_list = [x.squeeze().tolist() for x in y_pred_list]
        y_targ_list = [x.squeeze().tolist() for x in y_targ_list]
        y_pred_list = [x for sublist in y_pred_list for x in sublist]
        y_targ_list = [x for sublist in y_targ_list for x in sublist]

        print(classification_report(y_targ_list, y_pred_list))

    train_loss = train_loss / len(train_loader.dataset)
    test_loss = test_loss / len(test_loader.dataset)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    print(f"Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s")
    print("\tTraining Loss: {:.6f} \Test Loss: {:.6f}".format(train_loss, test_loss))
    if test_loss <= test_min_loss:
        print("Test loss decreased ({:.6f} --> {:.6f}). Saving model...\n".format(test_min_loss, test_loss))
        torch.save(model_bilstm.state_dict(), "./models/bilstm_bert_model.pt")
        test_min_loss = test_loss
    print("="*70)

              precision    recall  f1-score   support

           0       0.90      0.95      0.93      1677
           1       0.32      0.18      0.23       206

    accuracy                           0.87      1883
   macro avg       0.61      0.57      0.58      1883
weighted avg       0.84      0.87      0.85      1883

Epoch: 01 | Epoch Time: 0m 22s
	Training Loss: 0.033636 \Test Loss: 0.036043
Test loss decreased (inf --> 0.036043). Saving model...

              precision    recall  f1-score   support

           0       0.92      0.87      0.90      1677
           1       0.26      0.35      0.30       206

    accuracy                           0.82      1883
   macro avg       0.59      0.61      0.60      1883
weighted avg       0.84      0.82      0.83      1883

Epoch: 02 | Epoch Time: 0m 22s
	Training Loss: 0.032285 \Test Loss: 0.031966
Test loss decreased (0.036043 --> 0.031966). Saving model...

              precision    recall  f1-score   support

           0      

In [35]:
y_pred_list = []
y_targ_list = []
model = Bi_LSTM_Network(EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM).to(device)
model.load_state_dict(torch.load("models/bilstm_bert_model.pt"))
model.eval()

with torch.no_grad():
    for inputs, target in test_loader:
        inputs, target = inputs.to(device), target.to(device)
        inputs = torch.squeeze(inputs, dim=1)
        y_test_pred = model(inputs)
        _, y_test_pred = torch.max(y_test_pred, 1)
        y_pred_tag = y_test_pred
        y_pred_list.append(y_pred_tag.cpu().numpy())
        y_targ_list.append(target.cpu().numpy())

y_pred_list = [x.squeeze().tolist() for x in y_pred_list]
y_targ_list = [x.squeeze().tolist() for x in y_targ_list]
y_pred_list = [x for sublist in y_pred_list for x in sublist]
y_targ_list = [x for sublist in y_targ_list for x in sublist]

print(classification_report(y_targ_list, y_pred_list))

              precision    recall  f1-score   support

           0       0.95      0.72      0.82      1677
           1       0.23      0.67      0.34       206

    accuracy                           0.72      1883
   macro avg       0.59      0.70      0.58      1883
weighted avg       0.87      0.72      0.77      1883



# Best Performance : GRU